In [ ]:
import string
import re
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
# cargar doc en memoria
def load_doc(filename):
 # abrir el archivo como de sólo lectura
 file = open(filename, 'r')
 # leer todo el texto
 text = file.read()
 # cerrar el archivo
 file.close()
 return text
# convertir doc en tokens limpios
def clean_doc(doc, vocab):
 # dividido en tokens por espacio en blanco
 tokens = doc.split()
 # prepare a regex para el filtrado de caracteres
 re_punc = re.compile('[%s]' % re.escape(string.punctuation))
 # eliminar la puntuación de cada palabra
 tokens = [re_punc.sub('', w) for w in tokens]
 # filtrar las tokens que no están en el vocabulario
 tokens = [w for w in tokens if w in vocab]
 tokens = ' '.join(tokens)
 return tokens
# cargar todos los documentos en un directorio
def process_docs(directory, vocab, is_train):
 documents = list()
 # revisar todos los archivos de la carpeta
 for filename in listdir(directory):
  # omita cualquier comentario en el juego de pruebas
  if is_train and filename.startswith('cv9'):
   continue
  if not is_train and not filename.startswith('cv9'):
   continue
  # crear la ruta completa del archivo a abrir
  path = directory + '/' + filename
  # cargar el documento
  doc = load_doc(path)
  # expediente limpio
  tokens = clean_doc(doc, vocab)
  # añadir a la lista
  documents.append(tokens)
 return documents
# cargar y limpiar un conjunto de datos
def load_clean_dataset(vocab, is_train):
 # cargar documentos
 neg = process_docs('txt_sentoken/neg', vocab, is_train)
 pos = process_docs('txt_sentoken/pos', vocab, is_train)
 docs = neg + pos
 # preparar las etiquetas
 labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
 return docs, labels
# instalar un tokenizador
def create_tokenizer(lines):
 tokenizer = Tokenizer()
 tokenizer.fit_on_texts(lines)
 return tokenizer
# codificar y rellenar documentos enteros
def encode_docs(tokenizer, max_length, docs):
 # codificación entera
 encoded = tokenizer.texts_to_sequences(docs)
 # secuencias de pads
 padded = pad_sequences(encoded, maxlen=max_length, padding='post')
 return padded
# definir el modelo
def define_model(vocab_size, max_length):
 model = Sequential()
 model.add(Embedding(vocab_size, 100, input_length=max_length))
 model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
 model.add(MaxPooling1D(pool_size=2))
 model.add(Flatten())
 model.add(Dense(10, activation='relu'))
 model.add(Dense(1, activation='sigmoid'))
 # compilar red
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 # resumir el modelo definido
 model.summary()
 plot_model(model, to_file='model.png', show_shapes=True)
 return model


# cargar el vocabulario
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())
# cargar datos de entrenamiento
train_docs, ytrain = load_clean_dataset(vocab, True)
# crear el tokenizador
tokenizer = create_tokenizer(train_docs)
# definir el tamaño del vocabulario
vocab_size = len(tokenizer.word_index) + 1
print('Tamaño del vocabulario: %d' % vocab_size)
# calcular la longitud máxima de la secuencia
max_length = max([len(s.split()) for s in train_docs])
print('Longitud Maxima: %d' % max_length)
# datos codificados
Xtrain = encode_docs(tokenizer, max_length, train_docs)
# definir modelo
model = define_model(vocab_size, max_length)
# red adecuada
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# guardar el modelo
